## Retired logistic Regression code

In [ ]:
#len(vocab) = 258
#len(triggers) = 108
#a context has dimensionality 125 + 258
#context_vec
#wd_idx1, wd_idx2, wd_idx3, vocab_idx_1, dots vocab_idx_258, trigger_idx_1, ... trigger_idx_258

#training data
#25614 (= 90% of num_events = num sentences in brown after filtering)
#test data
#2854 (= 10% of num_events = num sentences in brown after filtering)


ngram_offset = V
num_triggers = len(triggers)
dim = ngram_offset + num_triggers
encoder = {wd:i for (i,wd) in enumerate(vocab)}
trig_encoder = {wd:ti for (ti, wd) in enumerate(triggers)}

def process_corpus (sents,V,encoder,trig_encoder,batch_sz = 10_000):
    corpus = np.zeros((V,dim))
    #corpus = np.zeros((V,dim))
    token_ct,next_milestone = 0, batch_sz
    for sent in sents:
        token_ct = process_sent (sent,token_ct, corpus,encoder,trig_encoder)
        if token_ct > next_milestone:
            print(f"{token_ct} words processed")
            next_milestone += batch_sz
    return corpus

def process_sent (sent,token_ct,corpus,encoder,trig_encoder):
    triggers_in = []
    for i in range(len(sent)):
        wd_idx = encoder[sent[i]]
        corpus[wd_idx,:] += add_context_vector(sent[:i],sent[i], triggers_in,
                                                 encoder,trig_encoder)
    return token_ct + len(sent) 

def update_bigram_context (context,prev,encoder):
    if prev is not None:
        context[encoder[prev]] += 1

def update_trigger_context (context,t,trig_encoder):
    # Use trigger encoding plus ngram_offset
    context[trig_encoder[t]+ngram_offset] += 1
    
def add_context_vector (hist, w, triggers_in,encoder,trig_encoder):
    """
    For the first word of sent (hist = []) the values of the trigram feats wd_idx1, wd_idx2, wd_idx3,
    are -3,-2,-1, For secon d word of sent  -2,-1,idx where idx is the word idx of the first word
    in the sent.  For third word of sent: -1,idx1,idx2 where idx1 and isdx2 are the word 
    indexes of the first two words of sent.  Obvious
    """
    global dim
    context = np.zeros((dim,))
    if hist:
        prev = hist[-1] 
    else:
        prev = None
    if prev in triggers:
        triggers_in.append(hist[-1])
    for t in triggers_in:
        update_trigger_context(context,t,trig_encoder)
    update_bigram_context(context,prev,encoder)
    return context

In [ ]:

corpus = process_corpus (sents,V,encoder,trig_encoder)

In [1]:
import numpy as np


def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return numpy.allclose(a, a.T, rtol=rtol, atol=atol)

class Triangular2DArray ():
    
    print_width = 8
    lbrack = "|"
    rbrack = "|"
    
    def __init__ (self, A, verbose = False):
        self.m,self.n = A.shape
        assert self.m == self.n, "Only square arrays may be converted to Triangular2DArray"
        if verbose:
            zeroed_A = np.triu(A)
            print(zeroed_A)
        self.size = int((self.m*(self.m+1))/2)
        #print(self.size)
        self._a = np.zeros((self.size,),dtype=A.dtype) #np.ndarray.flatten(zeroed_A)
        ctr = 0
        for i in range(self.m):
            for j in range(i,self.m):
                #d = zeroed_A[i,j]
                #assert d>0,"zeroed data error"
                #print(i,j,d,i*self.n + j)
                if j>=i:
                    self._a[ctr] =  A[i,j]   #d
                    ctr += 1
        self.shape = A.shape
        #del A
        
    def __getitem__ (self,pair):
        i,j = pair
        assert j >= i, f"{i=} {j=} not appropriate for upper triangular array."
        assert i < self.m and j < self.n, f"{i=} {j=} not appropriate for {m}x{n} array."
        return self._a[self.get_idx(i,j)]
         
    def __setitem__ (self,pair,data):
        i,j = pair
        assert j <= i, f"{i=} {j=} not appropriate for upper triangular array."
        assert i < self.m and j < self.n, f"{i=} {j=} not appropriate for {m}x{n} array."
        self._a[self.get_idx(i,j)] = data

            
    def __repr__ (self):
        str_list = []
        for i in range(self.m):
            row = [self.lbrack]
            for j in range(self.n):
                if i > j:
                    d = 0
                else:
                    d = self.__getitem__((i,j))
                row.append (self.make_print_rep (d))
            row.append(self.rbrack)
            str_list.append("".join(row))
        return "\n".join(str_list)
            
    def make_print_rep (self, d):
        return str(d).rjust(self.print_width," ")
    
    def idx_ct_row (self, j):
        return sum(self.n-i for i in range(j+1))
    
    def get_idx (self, i,j):
        #i*self.n + j
        #print(i,j,self.idx_ct_row (i-1), self.idx_ct_row (i-1) +j
        #    )
        return self.idx_ct_row (i-1) +(j-i)

#A = np.arange(1,26).reshape((5,5))
#ta = Triangular2DArray(A)